In [33]:
import logging
import os
from typing import Any

import datasets

import transformer_document_embedding as tde

os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")  # Report only TF errors by default

import tensorflow as tf

logging.basicConfig(
    format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO
)

In [12]:
import importlib

importlib.reload(tde)

<module 'transformer_document_embedding' from '/home/dburian/docs/transformer_document_embedding/src/transformer_document_embedding/__init__.py'>

2022-12-21 11:10:29,620 : INFO : {"jsonrpc": "2.0", "method": "SyncRequest", "params": {"data": {"file_name": "/home/dburian/docs/transformer_document_embedding/notebooks/doc2vec_imdb.sync.py", "contents": "# ---\n# jupyter:\n#  jupytext:\n#   text_representation:\n#    extension: .py\n#    format_name: percent\n#    format_version: '1.3'\n#    jupytext_version: 1.3.4\n#  kernelspec:\n#   display_name: Python 3\n#   language: python\n#   name: python3\n# ---\n# %%\nimport logging\nimport os\nfrom typing import Any\n\nimport transformer_document_embedding as tde\n\nos.environ.setdefault(\"TF_CPP_MIN_LOG_LEVEL\", \"2\") # Report only TF errors by default\n\nimport tensorflow as tf\n\nlogging.basicConfig(\n  format=\"%(asctime)s : %(levelname)s : %(message)s\", level=logging.INFO\n)\n# %%\nimport importlib\n\nimportlib.reload(tde)\n\n# %%\nLOG_DIR = \"../results/notebooks/doc2vec_imdb\"\ntask = tde.tasks.IMDBClassification()\nmodel = tde.models.IMDBDoc2Vec(log_dir=LOG_DIR)\n# %%\nfor i, d

In [2]:
LOG_DIR = "../results/notebooks/doc2vec_imdb"
task = tde.tasks.IMDBClassification()
model = tde.models.IMDBDoc2Vec(log_dir=LOG_DIR)

2022-12-21 10:53:56,085 : INFO : using concatenative 8400-dimensional layer1
2022-12-21 10:53:56,087 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/c,d400,n5,hs,w10,mc5,s0.001,t6>', 'datetime': '2022-12-21T10:53:56.086992', 'gensim': '4.2.0', 'python': '3.10.8 (main, Nov  1 2022, 14:18:21) [GCC 12.2.0]', 'platform': 'Linux-6.0.11-1-MANJARO-x86_64-with-glibc2.36', 'event': 'created'}
2022-12-21 10:53:56,087 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dbow,d400,n5,hs,mc5,s0.001,t6>', 'datetime': '2022-12-21T10:53:56.087554', 'gensim': '4.2.0', 'python': '3.10.8 (main, Nov  1 2022, 14:18:21) [GCC 12.2.0]', 'platform': 'Linux-6.0.11-1-MANJARO-x86_64-with-glibc2.36', 'event': 'created'}


In [3]:
for i, doc in enumerate(task.train):
    if i < 3:
        print(doc)
    if 25000 < i < 25003:
        print(doc)
    if i > 25003:
        break

2022-12-21 10:53:58,497 : WARNING : Found cached dataset imdb (/home/dburian/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
2022-12-21 10:53:58,523 : WARNING : Loading cached processed dataset at /home/dburian/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-299cf9fbc579d252.arrow
2022-12-21 10:54:00,796 : WARNING : Found cached dataset imdb (/home/dburian/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
2022-12-21 10:54:00,820 : WARNING : Loading cached processed dataset at /home/dburian/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-ff884c04faccb420.arrow


{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [4]:
model._doc2vec.train(
    task.train,
    min_doc_id=25000,
    max_doc_id=100000 - 1,
    epochs=1,
)

2022-12-21 10:54:01,661 : INFO : collecting all words and their counts
2022-12-21 10:54:01,662 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2022-12-21 10:54:02,480 : INFO : PROGRESS: at example #10000, processed 2317452 words (2835978 words/s), 152756 word types, 0 tags
2022-12-21 10:54:03,333 : INFO : PROGRESS: at example #20000, processed 4663219 words (2750146 words/s), 242583 word types, 0 tags
2022-12-21 10:54:04,299 : INFO : PROGRESS: at example #30000, processed 7020384 words (2442625 words/s), 315459 word types, 0 tags
2022-12-21 10:54:05,204 : INFO : PROGRESS: at example #40000, processed 9356892 words (2583943 words/s), 380568 word types, 0 tags
2022-12-21 10:54:06,044 : INFO : PROGRESS: at example #50000, processed 11731695 words (2830278 words/s), 441275 word types, 0 tags
2022-12-21 10:54:06,879 : INFO : PROGRESS: at example #60000, processed 14053055 words (2778745 words/s), 497123 word types, 0 tags
2022-12-21 10:54:07,869 : INFO 

2022-12-21 10:54:45,848 : INFO : EPOCH 0 - PROGRESS: at 4.54% examples, 26723 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:54:47,000 : INFO : EPOCH 0 - PROGRESS: at 4.71% examples, 26370 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:54:48,297 : INFO : EPOCH 0 - PROGRESS: at 4.93% examples, 26188 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:54:49,306 : INFO : EPOCH 0 - PROGRESS: at 5.15% examples, 26314 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:54:50,334 : INFO : EPOCH 0 - PROGRESS: at 5.38% examples, 26414 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:54:51,548 : INFO : EPOCH 0 - PROGRESS: at 5.61% examples, 26343 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:54:52,599 : INFO : EPOCH 0 - PROGRESS: at 5.84% examples, 26421 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:54:53,666 : INFO : EPOCH 0 - PROGRESS: at 6.05% examples, 26472 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:54:54,740 : INFO : EPOCH 0 - PROGRESS: at 6.30% examples, 26765 words/s, in_qsize 11, out_

2022-12-21 10:56:12,523 : INFO : EPOCH 0 - PROGRESS: at 22.66% examples, 27541 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:56:14,294 : INFO : EPOCH 0 - PROGRESS: at 23.03% examples, 27505 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:56:15,862 : INFO : EPOCH 0 - PROGRESS: at 23.35% examples, 27512 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:56:17,520 : INFO : EPOCH 0 - PROGRESS: at 23.71% examples, 27503 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:56:19,013 : INFO : EPOCH 0 - PROGRESS: at 24.03% examples, 27529 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:56:20,026 : INFO : EPOCH 0 - PROGRESS: at 24.29% examples, 27608 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:56:21,292 : INFO : EPOCH 0 - PROGRESS: at 24.53% examples, 27561 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:56:22,356 : INFO : EPOCH 0 - PROGRESS: at 24.77% examples, 27562 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:56:23,617 : INFO : EPOCH 0 - PROGRESS: at 25.05% examples, 27576 words/s, in_qsize

2022-12-21 10:57:45,029 : INFO : EPOCH 0 - PROGRESS: at 42.21% examples, 27759 words/s, in_qsize 12, out_qsize 0
2022-12-21 10:57:46,055 : INFO : EPOCH 0 - PROGRESS: at 42.50% examples, 27802 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:57:47,161 : INFO : EPOCH 0 - PROGRESS: at 42.69% examples, 27761 words/s, in_qsize 12, out_qsize 0
2022-12-21 10:57:48,336 : INFO : EPOCH 0 - PROGRESS: at 42.89% examples, 27746 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:57:49,779 : INFO : EPOCH 0 - PROGRESS: at 43.21% examples, 27766 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:57:51,277 : INFO : EPOCH 0 - PROGRESS: at 43.53% examples, 27780 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:57:52,328 : INFO : EPOCH 0 - PROGRESS: at 43.72% examples, 27747 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:57:53,496 : INFO : EPOCH 0 - PROGRESS: at 43.96% examples, 27734 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:57:55,031 : INFO : EPOCH 0 - PROGRESS: at 44.31% examples, 27745 words/s, in_qsize

2022-12-21 10:59:25,580 : INFO : EPOCH 0 - PROGRESS: at 63.41% examples, 27901 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:59:26,582 : INFO : EPOCH 0 - PROGRESS: at 63.71% examples, 27932 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:59:27,592 : INFO : EPOCH 0 - PROGRESS: at 63.90% examples, 27912 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:59:29,053 : INFO : EPOCH 0 - PROGRESS: at 64.16% examples, 27901 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:59:30,104 : INFO : EPOCH 0 - PROGRESS: at 64.46% examples, 27927 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:59:31,466 : INFO : EPOCH 0 - PROGRESS: at 64.74% examples, 27924 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:59:32,553 : INFO : EPOCH 0 - PROGRESS: at 64.95% examples, 27922 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:59:33,570 : INFO : EPOCH 0 - PROGRESS: at 65.17% examples, 27925 words/s, in_qsize 11, out_qsize 0
2022-12-21 10:59:34,587 : INFO : EPOCH 0 - PROGRESS: at 65.42% examples, 27930 words/s, in_qsize

2022-12-21 11:00:59,148 : INFO : EPOCH 0 - PROGRESS: at 83.71% examples, 28024 words/s, in_qsize 11, out_qsize 0
2022-12-21 11:01:00,628 : INFO : EPOCH 0 - PROGRESS: at 84.05% examples, 28032 words/s, in_qsize 11, out_qsize 0
2022-12-21 11:01:02,205 : INFO : EPOCH 0 - PROGRESS: at 84.41% examples, 28033 words/s, in_qsize 12, out_qsize 0
2022-12-21 11:01:03,721 : INFO : EPOCH 0 - PROGRESS: at 84.77% examples, 28039 words/s, in_qsize 11, out_qsize 0
2022-12-21 11:01:05,269 : INFO : EPOCH 0 - PROGRESS: at 85.10% examples, 28043 words/s, in_qsize 11, out_qsize 0
2022-12-21 11:01:06,744 : INFO : EPOCH 0 - PROGRESS: at 85.41% examples, 28049 words/s, in_qsize 11, out_qsize 0
2022-12-21 11:01:08,267 : INFO : EPOCH 0 - PROGRESS: at 85.75% examples, 28055 words/s, in_qsize 11, out_qsize 0
2022-12-21 11:01:09,798 : INFO : EPOCH 0 - PROGRESS: at 86.06% examples, 28059 words/s, in_qsize 11, out_qsize 0
2022-12-21 11:01:11,334 : INFO : EPOCH 0 - PROGRESS: at 86.43% examples, 28064 words/s, in_qsize

2022-12-21 11:02:22,896 : INFO : EPOCH 0 - PROGRESS: at 26.99% examples, 387843 words/s, in_qsize 11, out_qsize 1
2022-12-21 11:02:23,913 : INFO : EPOCH 0 - PROGRESS: at 30.16% examples, 390373 words/s, in_qsize 11, out_qsize 0
2022-12-21 11:02:24,945 : INFO : EPOCH 0 - PROGRESS: at 33.12% examples, 390077 words/s, in_qsize 12, out_qsize 0
2022-12-21 11:02:26,005 : INFO : EPOCH 0 - PROGRESS: at 36.39% examples, 390675 words/s, in_qsize 12, out_qsize 0
2022-12-21 11:02:27,020 : INFO : EPOCH 0 - PROGRESS: at 39.44% examples, 392026 words/s, in_qsize 11, out_qsize 1
2022-12-21 11:02:28,049 : INFO : EPOCH 0 - PROGRESS: at 42.63% examples, 392717 words/s, in_qsize 11, out_qsize 0
2022-12-21 11:02:29,054 : INFO : EPOCH 0 - PROGRESS: at 45.69% examples, 393547 words/s, in_qsize 11, out_qsize 0
2022-12-21 11:02:30,054 : INFO : EPOCH 0 - PROGRESS: at 48.79% examples, 394328 words/s, in_qsize 11, out_qsize 0
2022-12-21 11:02:31,065 : INFO : EPOCH 0 - PROGRESS: at 51.89% examples, 394877 words/s,

In [5]:
tf_ds = model._to_tf_dataset(task.train, training=True).shuffle(25000)

for batch in tf_ds.take(5):
    print(batch)

2022-12-21 11:02:46,998 : WARNING : Loading cached processed dataset at /home/dburian/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-d3d2c23422642dcc.arrow
2022-12-21 11:02:47,281 : WARNING : Parameter 'function'=<function IMDBDoc2Vec._to_tf_dataset.<locals>.<lambda> at 0x7fd4ff7a5900> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/25000 [00:00<?, ?ex/s]

(<tf.Tensor: shape=(2, 800), dtype=float32, numpy=
array([[-0.00331919,  0.00081138, -0.00329357, ..., -0.05986395,
        -0.15210591, -0.06205365],
       [ 0.00268728,  0.00534621,  0.00690543, ..., -0.047584  ,
        -0.01430116, -0.04755143]], dtype=float32)>, <tf.Tensor: shape=(2,), dtype=int64, numpy=array([0, 0])>)
(<tf.Tensor: shape=(2, 800), dtype=float32, numpy=
array([[ 0.00857443,  0.01514327,  0.0061665 , ..., -0.05915277,
        -0.10532252, -0.04429276],
       [-0.0006623 ,  0.0103007 ,  0.01685007, ..., -0.03570338,
        -0.05312823, -0.12384202]], dtype=float32)>, <tf.Tensor: shape=(2,), dtype=int64, numpy=array([0, 1])>)
(<tf.Tensor: shape=(2, 800), dtype=float32, numpy=
array([[ 3.00449575e-03,  9.32767987e-03,  3.95790965e-04, ...,
        -1.37519548e-02, -6.78614378e-02, -5.10153696e-02],
       [-2.39831628e-03,  1.62652024e-04,  2.06642468e-02, ...,
        -6.72813281e-02, -1.07971355e-01, -1.95203394e-01]], dtype=float32)>, <tf.Tensor: shape=(2,), dty

2022-12-21 11:06:56,786 : INFO : {"jsonrpc": "2.0", "method": "SyncRequest", "params": {"data": {"file_name": "/home/dburian/docs/transformer_document_embedding/notebooks/doc2vec_imdb.sync.py", "contents": "# ---\n# jupyter:\n#  jupytext:\n#   text_representation:\n#    extension: .py\n#    format_name: percent\n#    format_version: '1.3'\n#    jupytext_version: 1.3.4\n#  kernelspec:\n#   display_name: Python 3\n#   language: python\n#   name: python3\n# ---\n# %%\nimport logging\nimport os\nfrom typing import Any\n\nimport transformer_document_embedding as tde\n\nos.environ.setdefault(\"TF_CPP_MIN_LOG_LEVEL\", \"2\") # Report only TF errors by default\n\nimport tensorflow as tf\n\nlogging.basicConfig(\n  format=\"%(asctime)s : %(levelname)s : %(message)s\", level=logging.INFO\n)\n# %%\nLOG_DIR = \"../results/notebooks/doc2vec_imdb\"\ntask = tde.tasks.IMDBClassification()\nmodel = tde.models.IMDBDoc2Vec(log_dir=LOG_DIR)\n# %%\nfor i, doc in enumerate(task.train):\n  if i < 3:\n    prin

In [6]:
model._cls_head.fit(
    tf_ds, epochs=10, callbacks=[tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR)]
)

Epoch 1/10
12500/12500 [==============================] - 16s 797us/step - loss: 0.4194 - binary_accuracy: 0.8016
Epoch 2/10
12500/12500 [==============================] - 15s 757us/step - loss: 0.2740 - binary_accuracy: 0.8854
Epoch 3/10
12500/12500 [==============================] - 15s 747us/step - loss: 0.2264 - binary_accuracy: 0.9065
Epoch 4/10
12500/12500 [==============================] - 15s 751us/step - loss: 0.1955 - binary_accuracy: 0.9224
Epoch 5/10
12500/12500 [==============================] - 15s 748us/step - loss: 0.1744 - binary_accuracy: 0.9314
Epoch 6/10
12500/12500 [==============================] - 15s 746us/step - loss: 0.1565 - binary_accuracy: 0.9393
Epoch 7/10
12500/12500 [==============================] - 15s 771us/step - loss: 0.1456 - binary_accuracy: 0.9438
Epoch 8/10
12500/12500 [==============================] - 16s 840us/step - loss: 0.1370 - binary_accuracy: 0.9474
Epoch 9/10
12500/12500 [==============================] - 20s 1ms/step - loss: 0.1259 - 

2022-12-21 10:53:05,273 : INFO : {"jsonrpc": "2.0", "method": "SyncRequest", "params": {"data": {"file_name": "/home/dburian/docs/transformer_document_embedding/notebooks/doc2vec_imdb.sync.py", "contents": "# ---\n# jupyter:\n#  jupytext:\n#   text_representation:\n#    extension: .py\n#    format_name: percent\n#    format_version: '1.3'\n#    jupytext_version: 1.3.4\n#  kernelspec:\n#   display_name: Python 3\n#   language: python\n#   name: python3\n# ---\n# %%\nimport logging\nimport os\nfrom typing import Any\n\nimport transformer_document_embedding as tde\n\nos.environ.setdefault(\"TF_CPP_MIN_LOG_LEVEL\", \"2\") # Report only TF errors by default\n\nimport tensorflow as tf\n\nlogging.basicConfig(\n  format=\"%(asctime)s : %(levelname)s : %(message)s\", level=logging.INFO\n)\n# %%\nLOG_DIR = \"../results/notebooks/doc2vec_imdb\"\ntask = tde.tasks.IMDBClassification()\nmodel = tde.models.IMDBDoc2Vec(log_dir=LOG_DIR)\n# %%\nfor i, doc in enumerate(task.train):\n  if i < 3:\n    prin

In [6]:
test_predictions = model.predict(task.test)
results = task.evaluate(test_predictions)

2022-12-21 11:06:59,616 : WARNING : Found cached dataset imdb (/home/dburian/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
2022-12-21 11:06:59,669 : WARNING : Loading cached processed dataset at /home/dburian/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-072999855889cdf5.arrow


  0%|          | 0/25000 [00:00<?, ?ex/s]

12500/12500 [==============================] - 12s 919us/step


2022-12-21 11:07:31,834 : INFO : {"jsonrpc": "2.0", "method": "SyncRequest", "params": {"data": {"file_name": "/home/dburian/docs/transformer_document_embedding/notebooks/doc2vec_imdb.sync.py", "contents": "# ---\n# jupyter:\n#  jupytext:\n#   text_representation:\n#    extension: .py\n#    format_name: percent\n#    format_version: '1.3'\n#    jupytext_version: 1.3.4\n#  kernelspec:\n#   display_name: Python 3\n#   language: python\n#   name: python3\n# ---\n# %%\nimport logging\nimport os\nfrom typing import Any\n\nimport transformer_document_embedding as tde\n\nos.environ.setdefault(\"TF_CPP_MIN_LOG_LEVEL\", \"2\") # Report only TF errors by default\n\nimport tensorflow as tf\n\nlogging.basicConfig(\n  format=\"%(asctime)s : %(levelname)s : %(message)s\", level=logging.INFO\n)\n# %%\nLOG_DIR = \"../results/notebooks/doc2vec_imdb\"\ntask = tde.tasks.IMDBClassification()\nmodel = tde.models.IMDBDoc2Vec(log_dir=LOG_DIR)\n# %%\nfor i, doc in enumerate(task.train):\n  if i < 3:\n    prin

In [13]:
print(results)
tde.evaluation.save_results(results, LOG_DIR)

{'binary_crossentropy': 0.6931428, 'binary_accuracy': 0.50212}


2022-12-21 11:38:40,819 : INFO : {"jsonrpc": "2.0", "method": "SyncRequest", "params": {"data": {"file_name": "/home/dburian/docs/transformer_document_embedding/notebooks/doc2vec_imdb.sync.py", "contents": "# ---\n# jupyter:\n#  jupytext:\n#   text_representation:\n#    extension: .py\n#    format_name: percent\n#    format_version: '1.3'\n#    jupytext_version: 1.3.4\n#  kernelspec:\n#   display_name: Python 3\n#   language: python\n#   name: python3\n# ---\n# %%\nimport logging\nimport os\nfrom typing import Any\n\nimport transformer_document_embedding as tde\n\nos.environ.setdefault(\"TF_CPP_MIN_LOG_LEVEL\", \"2\") # Report only TF errors by default\n\nimport tensorflow as tf\n\nlogging.basicConfig(\n  format=\"%(asctime)s : %(levelname)s : %(message)s\", level=logging.INFO\n)\n# %%\nimport importlib\n\nimportlib.reload(tde)\n\n# %%\nLOG_DIR = \"../results/notebooks/doc2vec_imdb\"\ntask = tde.tasks.IMDBClassification()\nmodel = tde.models.IMDBDoc2Vec(log_dir=LOG_DIR)\n# %%\nfor i, d

In [27]:
task.train
a = task.train["label"]

In [30]:
a[3000]

"This is a bad movie. Not one of the funny bad ones either. This is a lousy bad one. It was actually painful to watch. The direction was awful,with lots of jumping around and the green and yellow hues used throughout the movie makes the characters look sickly. Keira Knightly was not convincing as a tough chick at all,and I cannot believe Lucy Liu and Mickey Rourke signed on for this criminal waste of celluloid. The script was terrible and the acting was like fingernails across a chalkboard. If you haven't seen it,don't. You are not missing anything and will only waste two hours of your life watching this drivel .I have seen bad movies before and even enjoyed them due to their faults. This one is just a waste of time."

2022-12-21 12:08:10,555 : INFO : {"jsonrpc": "2.0", "method": "SyncRequest", "params": {"data": {"file_name": "/home/dburian/docs/transformer_document_embedding/notebooks/doc2vec_imdb.sync.py", "contents": "# ---\n# jupyter:\n#  jupytext:\n#   text_representation:\n#    extension: .py\n#    format_name: percent\n#    format_version: '1.3'\n#    jupytext_version: 1.3.4\n#  kernelspec:\n#   display_name: Python 3\n#   language: python\n#   name: python3\n# ---\n# %%\nimport logging\nimport os\nfrom typing import Any\n\nimport transformer_document_embedding as tde\n\nos.environ.setdefault(\"TF_CPP_MIN_LOG_LEVEL\", \"2\") # Report only TF errors by default\n\nimport tensorflow as tf\n\nlogging.basicConfig(\n  format=\"%(asctime)s : %(levelname)s : %(message)s\", level=logging.INFO\n)\n# %%\nimport importlib\n\nimportlib.reload(tde)\n\n# %%\nLOG_DIR = \"../results/notebooks/doc2vec_imdb\"\ntask = tde.tasks.IMDBClassification()\nmodel = tde.models.IMDBDoc2Vec(log_dir=LOG_DIR)\n# %%\nfor i, d

In [31]:
data = task.train.filter(lambda doc: doc["label"] >= 0)
features_iter = model._doc2vec.predict(data)

  0%|          | 0/75 [00:00<?, ?ba/s]

2022-12-21 12:10:53,902 : INFO : {"jsonrpc": "2.0", "method": "SyncRequest", "params": {"data": {"file_name": "/home/dburian/docs/transformer_document_embedding/notebooks/doc2vec_imdb.sync.py", "contents": "# ---\n# jupyter:\n#  jupytext:\n#   text_representation:\n#    extension: .py\n#    format_name: percent\n#    format_version: '1.3'\n#    jupytext_version: 1.3.4\n#  kernelspec:\n#   display_name: Python 3\n#   language: python\n#   name: python3\n# ---\n# %%\nimport logging\nimport os\nfrom typing import Any\n\nimport datasets\n\nimport transformer_document_embedding as tde\n\nos.environ.setdefault(\"TF_CPP_MIN_LOG_LEVEL\", \"2\") # Report only TF errors by default\n\nimport tensorflow as tf\n\nlogging.basicConfig(\n  format=\"%(asctime)s : %(levelname)s : %(message)s\", level=logging.INFO\n)\n# %%\nimport importlib\n\nimportlib.reload(tde)\n\n# %%\nLOG_DIR = \"../results/notebooks/doc2vec_imdb\"\ntask = tde.tasks.IMDBClassification()\nmodel = tde.models.IMDBDoc2Vec(log_dir=LOG_D

In [35]:
data = data.map(
    lambda _: {"features": next(features_iter)},
    keep_in_memory=True,
    load_from_cache_file=False,
)

  0%|          | 0/25000 [00:00<?, ?ex/s]

2022-12-21 12:17:36,021 : INFO : {"jsonrpc": "2.0", "method": "SyncRequest", "params": {"data": {"file_name": "/home/dburian/docs/transformer_document_embedding/notebooks/doc2vec_imdb.sync.py", "contents": "# ---\n# jupyter:\n#  jupytext:\n#   text_representation:\n#    extension: .py\n#    format_name: percent\n#    format_version: '1.3'\n#    jupytext_version: 1.3.4\n#  kernelspec:\n#   display_name: Python 3\n#   language: python\n#   name: python3\n# ---\n# %%\nimport logging\nimport os\nfrom typing import Any\n\nimport datasets\n\nimport transformer_document_embedding as tde\n\nos.environ.setdefault(\"TF_CPP_MIN_LOG_LEVEL\", \"2\") # Report only TF errors by default\n\nimport tensorflow as tf\n\nlogging.basicConfig(\n  format=\"%(asctime)s : %(levelname)s : %(message)s\", level=logging.INFO\n)\n# %%\nimport importlib\n\nimportlib.reload(tde)\n\n# %%\nLOG_DIR = \"../results/notebooks/doc2vec_imdb\"\ntask = tde.tasks.IMDBClassification()\nmodel = tde.models.IMDBDoc2Vec(log_dir=LOG_D

In [36]:
try:
    print(next(features_iter))
except StopIteration:
    print("no more features")

no more features


2022-12-21 12:18:57,031 : INFO : {"jsonrpc": "2.0", "method": "SyncRequest", "params": {"data": {"file_name": "/home/dburian/docs/transformer_document_embedding/notebooks/doc2vec_imdb.sync.py", "contents": "# ---\n# jupyter:\n#  jupytext:\n#   text_representation:\n#    extension: .py\n#    format_name: percent\n#    format_version: '1.3'\n#    jupytext_version: 1.3.4\n#  kernelspec:\n#   display_name: Python 3\n#   language: python\n#   name: python3\n# ---\n# %%\nimport logging\nimport os\nfrom typing import Any\n\nimport datasets\n\nimport transformer_document_embedding as tde\n\nos.environ.setdefault(\"TF_CPP_MIN_LOG_LEVEL\", \"2\") # Report only TF errors by default\n\nimport tensorflow as tf\n\nlogging.basicConfig(\n  format=\"%(asctime)s : %(levelname)s : %(message)s\", level=logging.INFO\n)\n# %%\nimport importlib\n\nimportlib.reload(tde)\n\n# %%\nLOG_DIR = \"../results/notebooks/doc2vec_imdb\"\ntask = tde.tasks.IMDBClassification()\nmodel = tde.models.IMDBDoc2Vec(log_dir=LOG_D